In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("../input/women-entrepreneurship-and-labor-force/Dataset3.csv",sep=';')
data.style.background_gradient(cmap='YlGnBu', low=0, high=0, axis=0, subset=None)

******The Global Entrepreneurship Index (GEI)** is an economic activity index compiled by US-based The Global Entrepreneurship and Development Institute, which looks at how individual countries across the world allocate resources to promoting entrepreneurship, if indeed they do.
The Global Entrepreneurship Index is a composite indicator of the health of the entrepreneurship ecosystem in a given country.The GEI measures both the quality of entrepreneurship and the extent and depth of the supporting entrepreneurial ecosystem.

**Women entrepreneurs** may be defined as the women or a group of women who initiate, organise and operate a business enterprise. Women are expected to innovate, imitate or adopt an economic activity to be called women entrepreneurs.

**Why does entrepreneurship matter? How do entrepreneurs contribute?**


Entrepreneurs improve economies and people’s lives by creating jobs, developing new solutions to problems, creating technology that improves efficiency, and exchanging ideas globally. Many of the conditions that help entrepreneurs also help the economy as a whole, providing even broader gains from supporting entrepreneurship. 


![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQRPT84yt85J9Bji1J8x3Bcwj8Oin_FuDJZnQ&usqp=CAU)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data_copy=data.copy()

In [ ]:
category=[]
for col in data_copy.columns:
    if data[col].dtypes==object:
        category.append(col)

In [ ]:
data_encode=pd.get_dummies(data_copy,category,dtype=np.uint8)

In [ ]:
data_encode.head()

In [ ]:

from scipy.stats import shapiro

stat, p = shapiro(data_encode)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

In [ ]:
data.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
fig,ax=plt.subplots(figsize=(12,8))
sns.heatmap(data.corr(),annot=True)
plt.show()

It's observed that enterpreneurship index and women enterpreneurship index are strongly correlated as well as positively correlated.Also Female Labor Force Participation Rate and Women enterpreneurship is positively correlated.As expected inflation rate is negatively correlated with both enterpreneurship index as well as women enterpreneurship index.

In [ ]:
data['Country'].nunique()

In [ ]:
data['Level of development'].value_counts()

In [ ]:
fig = px.bar(data,
        x = "Level of development",
        y = "Women Entrepreneurship Index",
        color_discrete_sequence=["magenta"]
        
    )
    
fig.show()

In [ ]:
fig = px.bar(data,
        x = "European Union Membership",
        y = "Women Entrepreneurship Index",
        color_discrete_sequence=["red"]
        
    )
    
fig.show()

In [ ]:
import plotly.graph_objects as go
country_fig = go.Figure(data=[go.Pie(labels=data['Country'],
                             values=data['Women Entrepreneurship Index'],
                             hole=.7,
                             title = 'Count by Country',
                             marker_colors = px.colors.sequential.Greens_r,
                            )
                     ])
country_fig.update_layout(title = 'Women Enterpreneurship Index % by Country')
country_fig.show()


Australia has highest percentage of women enterpreneurship index.


India has lowest percentage of women enterpreneurship index.

In [ ]:
import plotly.express as px

fig = px.scatter(data, x="Female Labor Force Participation Rate", y="Women Entrepreneurship Index", color="Level of development",
                 size='Entrepreneurship Index', hover_data=['European Union Membership'])
fig.show()

In [ ]:
fig = px.scatter(
    data, x='Inflation rate', y='Women Entrepreneurship Index', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue'
)
fig.show()

In [ ]:
fig = px.violin(data, y="Women Entrepreneurship Index", x="Currency", color="Level of development", box=True, points="all",
          hover_data=data.columns)
fig.show()

Are European Union membership variable and development variable independent from each other?

Null Hypothesis-European Union membership variable and development variable are independent from each other.


Alternate Hypothesis-European Union membership variable and development variable are not  independent from each other.

In [ ]:
crosstab = pd.crosstab(data['European Union Membership'], data['Level of development'])

crosstab.style.background_gradient(cmap='YlGnBu', low=0, high=0, axis=0, subset=None)

In [ ]:
from scipy import stats

In [ ]:
stats.chi2_contingency(crosstab)

The first value (26.222328629032262) is the Chi-square value, followed by the p-value (3.042837837948524e-07), then comes the degrees of freedom (1), and lastly it outputs the expected frequencies as an array.

![](https://www.mun.ca/biology/scarr/IntroPopGen-Table-D-01-smc.jpg)

since for degree 1 and chi square value 26.222328629032262 we get probability less than 0.01 i.e less than alpha or level of significance(0.05) so we reject the null hypothesis.And Hence it is proved that
European Union membership variable and development variable are not independent from each other.

Do the Women Entrepreneurship Index and Global Entrepreneurship Index values show a statistically significant difference between the countries that are members of the European Union and not? (Method Mann-Whitney U)

H0-Women Entrepreneurship Index and Global Entrepreneurship Index values show a statistically significant difference between the countries that are members of the European Union and not.



Ha-Women Entrepreneurship Index and Global Entrepreneurship Index values do not show a statistically significant difference between the countries that are members of the European Union and not.

In [ ]:
data_eu=data[data['European Union Membership']=='Member']
data_not_eu=data[data['European Union Membership']=='Not Member']

In [ ]:
def mann_whitney_u_test(distribution_1, distribution_2):
    """
    Perform the Mann-Whitney U Test, comparing two different distributions.
    Args:
       distribution_1: List. 
       distribution_2: List.
    Outputs:
        u_statistic: Float. U statisitic for the test.
        p_value: Float.
    """
    u_statistic, p_value = stats.mannwhitneyu(distribution_1, distribution_2)
    return u_statistic, p_value


#### MAIN FUNCTION ####
#Perform the Mann-Whitney U Test on the two distributions
mann_whitney_u_test(list(data_eu['Women Entrepreneurship Index']), list(data_not_eu['Women Entrepreneurship Index']))


since p-value<0.05 ,so H0 is rejected.Hence,it is proved that Women Entrepreneurship Index and Global Entrepreneurship Index values do not show a statistically significant difference between the countries that are members of the European Union and not

In [ ]:
mann_whitney_u_test(list(data_eu['Entrepreneurship Index']), list(data_not_eu['Entrepreneurship Index']))

Is there a statistically significant relationship between Women's Entrepreneurship Index and Global Entrepreneurship Index values? 

In [ ]:
data[['Women Entrepreneurship Index','Entrepreneurship Index']].corr(method='spearman').iloc[0,1]

There  is a statistically significant relationship between Women's Entrepreneurship Index and Global Entrepreneurship Index values.

# **If You Liked It Please Upvote!!**

<img src="https://media.tenor.com/images/716f637338be29904408274b45c6dabd/tenor.gif">